In [11]:
import os
import pandas as pd
from pandas import Series, DataFrame

def get_data(path,province):
    table_name=province+'通用表.xls'
    if not os.path.exists(path+province+'通用表.xls'):
        table_name=province+'通用表.xlsx'   
    format_data=pd.read_excel(path+table_name,sheet_name=None)
    file_list=os.listdir(path)
    file_list.remove(table_name)
    unformat_data=pd.DataFrame(columns=[])
    for file in file_list:
        table_path=path+file
        tep_data=pd.read_excel(table_path)
        length=tep_data.shape[0]
        tep_data=tep_data[2:length-2]
        tep_data['指标名称']=tep_data['指标名称'].dt.strftime('%Y')
        tep_data=tep_data.set_index('指标名称')
        tep_data.index.names = ['年份']
        unformat_data=pd.concat([unformat_data,tep_data],axis=1)
    return format_data,unformat_data
        
        

def get_column_name(i):
    i=i.split('（',1)
    i=i[0]
    i=i.split('(',1)
    i=i[0]
    return i
def get_split_column_name(i):
    i=i.split(':')
    return i

def condition_judge(c,d):
    if get_column_name(c) =='GDP' and get_column_name(c) in get_split_column_name(d) :
        a=['第一产业','第二产业','第三产业']
        for i in a :
            if i in get_split_column_name(d):
                return False
        return True
    elif get_column_name(c) in get_split_column_name(d) :
        return True
    else:
        return False
                
            
    
path=input("请输入文件所在目录，如E:/工作进度/wind/,按回车结束输入：")
province=input("请输入省份，如安徽省,按回车结束输入：")
if  os.path.exists(path+province+'最终表.xlsx'):
    os.remove(path+province+'最终表.xlsx')
total_format_data,unformat_data=get_data(path=path,province=province)
citys=list(total_format_data.keys())
for i in citys:
    format_data=DataFrame(total_format_data[i])
    format_data['年份']=format_data['年份'].astype(str)
    location=format_data['地区'].map(lambda s: s.strip()).to_list()
    year=format_data['年份'].map(lambda s:  s.strip()).to_list()
    tem_index=zip(location,year)
    index_name=[]
    for j in tem_index :
        a=j[0]
        b=j[1]
        index_name.append(a+b)
    format_data['index']=index_name
    format_data=format_data.set_index('index')
    columns_name=format_data.columns.values.tolist()
    un_index_name=unformat_data._stat_axis.values.tolist()
    un_columns_name=unformat_data.columns.values.tolist()
    for a in index_name:
        for b in un_index_name:
            if a[-4:]==b:
                for c in columns_name:
                    for d in un_columns_name:
                        if a[0:len(a)-4] in get_split_column_name(d) and condition_judge(c,d):
                            format_data.loc[a,c]=unformat_data.loc[b,d]
                break
    format_data['地区']=format_data['地区'].map(str.strip)
    if not os.path.exists(path+province+'最终表.xlsx'):
        format_data.to_excel(path+province+'最终表.xlsx', sheet_name=i,index=False)
    else:    
        with pd.ExcelWriter(path+province+'最终表.xlsx',mode='a',engine='openpyxl') as writer:  
            format_data.to_excel(writer, sheet_name=i,index=False)

请输入文件所在目录，如E:/工作进度/wind/,按回车结束输入： E:/工作进度/test/
请输入省份，如安徽省,按回车结束输入： 安徽省
